# <a>PyTorch Autograd</a>

Uma das características mais interessantes do PyTorch é sua capacidade de calcular automaticaente as derivadas parciais e armazená-las nos tensores. Com o Autograd isso é possível. 

Apesar desse cálculo todo ficar "por debaixo dos panos", é interessante conhecer um pouco para entender a "mágica" de como a biblioteca facilita o uso do gradiente descendente para otimização de funções, muito utilizadas para treinamento de redes neurais, por exemplo.

Nos baseamos [nesse vídeo](https://www.youtube.com/watch?v=Poc0X5fS9us) do mestre dos mestres Abhishek Thakur para criar esse notebook.

Mas antes, vamos ver um pouquinho da integração dos tensores do PyTorch com o Numpy e como utilizar GPU para armazenar os tensores.

In [1]:
import numpy as np
import torch

In [2]:
# Bom e velho np array
array_np = np.array([[1,2],[3,4]])
array_np

array([[1, 2],
       [3, 4]])

In [4]:
array_np.shape

(2, 2)

In [5]:
# Convertendo de duas formas diferentes
tensor_pt = torch.tensor(array_np)
tensor_pt

tensor([[1, 2],
        [3, 4]], dtype=torch.int32)

In [6]:
tensor_pt = torch.from_numpy(array_np)
tensor_pt

tensor([[1, 2],
        [3, 4]], dtype=torch.int32)

In [7]:
type(tensor_pt)

torch.Tensor

In [8]:
# A volta também é fácil
tensor_pt.numpy()

array([[1, 2],
       [3, 4]])

In [9]:
type(tensor_pt.numpy())

numpy.ndarray

In [10]:
tensor_pt.size()

torch.Size([2, 2])

### <a> E GPU? Temos uma fuleirinha aqui pra testar, pelo menos! </a>

Lembrando que temos que estar com ambiente NVIDIA no SO bem certinho, com CUDA drivers e [cuDNN](https://docs.nvidia.com/deeplearning/cudnn/install-guide/index.html)

In [11]:
# Primeiro testando se tem GPU disponível
torch.cuda.is_available()

True

In [12]:
# Localizando a(s) GPU(s) disponível(eis)
plaquinha_fueba = torch.device('cuda') # ou cuda:0, cuda:1...
plaquinha_fueba

device(type='cuda')

In [14]:
# foi criado por padrão na cpu
tensor_pt.device

device(type='cpu')

In [15]:
tensor_pt = tensor_pt.to(plaquinha_fueba)
tensor_pt

tensor([[1, 2],
        [3, 4]], device='cuda:0', dtype=torch.int32)

In [16]:
# Pra jogar de volta pra CPU
cpu = torch.device('cpu')
tensor_pt = tensor_pt.to(cpu)
tensor_pt.device

device(type='cpu')

In [17]:
tensor_pt = tensor_pt.cuda()
tensor_pt

tensor([[1, 2],
        [3, 4]], device='cuda:0', dtype=torch.int32)

In [18]:
novo_tensor = torch.tensor(np.linspace(0., 1., 30), device=plaquinha_fueba)
novo_tensor

tensor([0.0000, 0.0345, 0.0690, 0.1034, 0.1379, 0.1724, 0.2069, 0.2414, 0.2759,
        0.3103, 0.3448, 0.3793, 0.4138, 0.4483, 0.4828, 0.5172, 0.5517, 0.5862,
        0.6207, 0.6552, 0.6897, 0.7241, 0.7586, 0.7931, 0.8276, 0.8621, 0.8966,
        0.9310, 0.9655, 1.0000], device='cuda:0', dtype=torch.float64)

### <a> E o Autograd??? </a>

In [19]:
horas_estudo = torch.tensor(5.) # pontinho pra dizer que queremos armazenar como float
horas_happy_hour = torch.tensor(6.)

horas_estudo, horas_happy_hour

(tensor(5.), tensor(6.))

In [20]:
# requires_grad informa que queremos calcular armazenar os gradientes para esse tensor
# a partir dessa flag, tudo que fizermos de operações com esses tensores
# será armazenado para fins de backpropagation
horas_estudo = torch.tensor(5., requires_grad=True) 
horas_happy_hour = torch.tensor(6., requires_grad=True)

horas_estudo, horas_happy_hour

(tensor(5., requires_grad=True), tensor(6., requires_grad=True))

In [21]:
nota_prova = horas_estudo ** 3 - horas_happy_hour ** 2

In [22]:
nota_prova

tensor(89., grad_fn=<SubBackward0>)

In [23]:
nota_prova.grad_fn.next_functions

((<PowBackward0 at 0x2a7a3473988>, 0), (<PowBackward0 at 0x2a7a34737c8>, 0))

In [24]:
nota_prova.grad_fn.next_functions[0][0]

In [25]:
nota_prova.grad_fn.next_functions[1][0]

In [26]:
nota_prova.grad_fn.next_functions[0][0].next_functions

((<AccumulateGrad at 0x2a7a3457ac8>, 0),)

In [27]:
nota_prova.grad_fn.next_functions[1][0].next_functions

((<AccumulateGrad at 0x2a7a344fa48>, 0),)

In [28]:
# Derivadas parciais: nota pelas horas de estudo
# d nota/d estudo = 3 * horas_estudo ** 2
3 * 5 ** 2

75

In [29]:
# Derivadas parciais: nota pelas horas de cachaç... ops, happy hour
# d nota/d cachaça = - 2 * horas_happy_hour ** 1
-2 * 6

-12

In [30]:
# Ainda não calculamos os gradientes com autograd
horas_estudo.grad, horas_happy_hour.grad

(None, None)

In [31]:
# Mágica do Autograd
nota_prova.backward()

In [32]:
# Agora sim
horas_estudo.grad, horas_happy_hour.grad

(tensor(75.), tensor(-12.))